In [208]:
import numpy as np
import pandas as pd
# import csv
# import matplotlib.cm as cm
# import matplotlib.mlab as mlab
# import matplotlib.pyplot as plt
# import scipy as sp
import scipy.io as sio
# import scipy.stats as stats
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import Imputer
from math import log2

In [2]:
SPAM = 'dist/spam_data.mat'
TRAIN_CENSUS = 'hw5_census_dist/train_data.csv'
TRAIN_TITANIC = 'hw5_titanic_dist/titanic_training.csv'

In [18]:
spam_data = sio.loadmat(SPAM)
census_tdata = pd.read_csv(TRAIN_CENSUS)
titanic_tdata = pd.read_csv(TRAIN_TITANIC)

In [187]:
"""Removed ticket and cabin features"""
titanic_tdata_filtered = titanic_tdata.drop('ticket', 1)
titanic_tdata_filtered = titanic_tdata_filtered.drop('cabin', 1)

"""Row 707 is empty in titanic_training.csv"""
titanic_tdata_filtered = titanic_tdata_filtered.dropna(how='all')

"""Map categories to binary variables [1(a)]"""
tv = DictVectorizer()
vectorized_titanic = tv.fit_transform(titanic_tdata_filtered.to_dict(orient='records'))

"""Replaced unknown values with mean of features [2(b)]"""
timp = Imputer()
imputed_titanic = timp.fit_transform(vectorized_titanic)

ttdata = pd.DataFrame(data=imputed_titanic.toarray(), columns=tv.get_feature_names(), dtype='object')
# ttlabels = ttdata['survived']
# ttdata = ttdata.drop('survived', 1)

In [168]:
"""Map categories to binary variables [1(a)]"""
cv = DictVectorizer()
vectorized_census = cv.fit_transform(census_tdata.to_dict(orient='records'))

"""Replaced unknown values with mean of features [2(b)]"""
cimp = Imputer()
imputed_census = cimp.fit_transform(vectorized_census)

ctdata = pd.DataFrame(data=imputed_census.toarray(), columns=cv.get_feature_names(), dtype='object')
# ctlabels = ctdata['label']
# ctdata = ctdata.drop('label', 1)

In [145]:
stdata = spam_data['training_data']
stlabels = spam_data['training_labels']

# Decision Tree Abstraction
## Class InternalNode
* ### State
    * Node left, right
    * split_feature [split rule]
    * split_value [split rule]
* ### Methods
    * `predict(data)`
        * given a data point, chooses left or right child based on the split rule
        * traverses starting from this node
    * `is_leaf() { return False }`

## Class LeafNode
* ### State
    * label
* ### Methods
    * `is_leaf() { return True }`

## Class DecisionTree
* ### State
    * root
* ### Methods
    * `impurity(left_label_hist, right_label_hist)`
        * calculates the entropy of a split
    * `segmenter(data, labels)`
        * finds the best split rule using impurity()
        * many different types of segmenters
    * `train(train_data, train_labels, depth_limited)`
        * grows the decision tree
        * uses segmenter to find the best splits
    * `predict(data)`
        * given a data point, traverses the tree starting at the root
    

In [209]:
class LeafNode:
    def __init__(self, label):
        self.label = label
        
    def predict(data):
        return self.label
    
    def is_leaf(self):
        return True
    
class InternalNode:
    def __init__(self, left, right, split_feature, split_value):
        self.left, self.right = left, right
        self.split_feature = split_feature 
        self.split_value = split_value
    
    def predict(data):
        """IMPLEMENT ME"""
        
    def is_leaf(self):
        return False
        
class TitanicDecisionTree:
    def __init__(self, root=None):
        self.root = root
        
    def impurity(self, C, D, nC, nD):
        c, d = nC - C, nD - D
        lefte  = C * log2(C / (C + D)) / sumCD + D * log2(D / (C + D))
        righte = c * log2(c / (c + d)) / sumcd + d * log2(d / (c + d))
        return -(lefte + righte) / (nC + nD)
        
    def count(self, data, label):
        nC, nD = 0, 0
        for lbl in data[label]:
            if lbl == 0:
                nC += 1
            else:
                nD += 1
        return nC, nD
        
    def segmenter(self, data, label):
        """
        For splits on a feature f with a value v, 
            left will have samples with f values strictly less than v
            right will have samples with f values greater than or equal to v 
        """
        nC, nD = self.count(data)
        min_entropy, splitf, splitv = float('inf'), None, None
        features = data.drop(label, 1).axes[1]
        for f in features:
            sorted_data_by_f = data.sort_values(f, axis=1)
            i, C, D = 0, 0, 0
            for beta in sorted_data_by_f.drop_duplicates(subset=f, keep='first'):
                while sorted_data_by_f[f][i] != beta:
                    if data[label][i] == 0:
                        C += 1
                    else:
                        D += 1
                    i += 1
                entropy = self.impurity(C, D, nC, nD)
                if entropy < min_entropy:
                    min_entropy, splitf, splitv = entropy, f, beta
            # no need to check split on max(beta)+1 since it's the same as min(beta)
            # i.e all elements are on one split
                    
        left = 
        right = 
        return left, right, splitf, splitv
                    
                    
        
    def train(self, train_data, label, depth_limited=float('inf')):
        labels = set(train_data[label])
        if depth_limited == 0 or len(labels) == 1:
            return LeafNode(labels.pop())
        left_data, right_data, split_feature, split_value = self.segmenter(train_data, label)
        left = self.train(left_data, depth_limited - 1)
        right = self.train(right_data, depth_limited - 1)
        return new InternalNode(left, right, split_feature, split_value)
        
    def predict(self, data):
        return self.root.predict(data)

SyntaxError: invalid syntax (<ipython-input-209-1d5df68218ae>, line 67)

In [207]:
# print(ttdata.iloc[0])
print(ttdata['survived'][0])
print(len(ttdata))
print(ttdata.shape[0])
print(ttdata.iloc[705])
print(set(ttdata['survived']), '\n')
s = set(ttdata.axes[1])
# for f in ttdata.drop('survived', 1).axes[1]:
#     print(f, type(f))
    
features = ttdata.drop('survived', 1).axes[1]
for f in features:
    print(f)
    sorted_data_by_f = ttdata.sort_values(f)
    print(sorted_data_by_f.iloc[-1])
    break;

0.0
999
999
age           30.2594
embarked            0
embarked=C          0
embarked=Q          0
embarked=S          1
fare             8.05
parch               0
pclass              3
sex=female          0
sex=male            1
sibsp               0
survived            0
Name: 705, dtype: object
{0.0, 1.0} 

age
age           80
embarked       0
embarked=C     0
embarked=Q     0
embarked=S     1
fare          30
parch          0
pclass         1
sex=female     0
sex=male       1
sibsp          0
survived       1
Name: 475, dtype: object


In [205]:
raw_data = {'first_name': ['Jason', 'Jason', 'Tina', 'Jake', 'Amy', 'Zach'],
        'last_name': ['Miller', 'Miller', 'Ali', 'Milner', 'Cooze', 'Miller'],
        'age': [42, 42, 36, 24, 73, 4],
        'preTestScore': [4, 4, 31, 2, 3, 100],
        'postTestScore': [25, 25, 57, 62, 70, 100]}
df = pd.DataFrame(raw_data, columns = ['first_name', 'last_name', 'age', 'preTestScore', 'postTestScore'])
print(df)
print(df.drop_duplicates(subset='first_name', keep='first'))

  first_name last_name  age  preTestScore  postTestScore
0      Jason    Miller   42             4             25
1      Jason    Miller   42             4             25
2       Tina       Ali   36            31             57
3       Jake    Milner   24             2             62
4        Amy     Cooze   73             3             70
5       Zach    Miller    4           100            100
  first_name last_name  age  preTestScore  postTestScore
0      Jason    Miller   42             4             25
2       Tina       Ali   36            31             57
3       Jake    Milner   24             2             62
4        Amy     Cooze   73             3             70
5       Zach    Miller    4           100            100
